In [5]:
import sys
import os
from pathlib import Path

# importディレクトリの追加
# sys.path.append(os.path.join(os.path.dirname(__file__), '..'))
print(sys.path)

# プロキシの設定
# os.environ['HTTP_PROXY'] = ''
# os.environ['HTTPS_PROXY'] = ''

%matplotlib inline

['/home/y-katayama/notebooks', '/usr/lib/python38.zip', '/usr/lib/python3.8', '/usr/lib/python3.8/lib-dynload', '', '/home/y-katayama/venv/pt1.7/lib/python3.8/site-packages']


In [1]:
!nvidia-smi

Tue Dec  6 16:04:40 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.60.13    Driver Version: 525.60.13    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  On   | 00000000:01:00.0 Off |                  N/A |
| 40%   32C    P8    17W / 184W |   1076MiB /  8192MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA GeForce ...  On   | 00000000:03:00.0 Off |                  N/A |
| 40%   

In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda

training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
            nn.ReLU()
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork()

100.0%

Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw


111.0%

Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw


159.1%/home/y-katayama/venv/pt1.7/lib/python3.8/site-packages/torchvision/datasets/mnist.py:480: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:141.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw
Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw
Processing...
Done!


# 最適化ループ

In [6]:
learning_rate = 1e-3
batch_size = 64
epochs = 5

- 最適化ループの1回のイテレーションは, エポックと呼ばれる
- 各エポックは2種類のループから構成される

  - 訓練ループ
  - 検証/テストループ
  
## 損失関数: Loss function

- 損失関数はモデルが推論した結果と, 実際の正解の誤差の大きさを測定する関数
- 損失を計算するためには, 入力データに対するモデルの推論結果を求めて, その値と正解ラベルとの違いを比較する

- 一般的な損失関数は回帰: `nn.MSELoss(Mean Square Error)`, 分類: `nn.NLLoss(Negative Log Likelihood)`が使用される
- `nn.CrossEntropyLoss`は`nn.LogSoftmax`と`nn.NLLLoss`を結合した損失関数
- モデルが出力するlogit値を`nn.CrossEntropyLoss()`に与えて正規化し, 予測誤差を求める


In [7]:
loss_fn = nn.CrossEntropyLoss()

## 最適化アルゴリズム

- 最適化アルゴの詳細については省略
- https://pytorch.org/docs/stable/optim.html

- 訓練ループ内ではoptimizerは以下のように動作する

  1. `optimizer.zero_grad()`を実行し, モデルパラメータの勾配をリセットする
  2. `loss.backwards()`を実行し, backpropを行う
  3. optimizer.step()を実行し, 各パラメータの勾配を使用してパラメータの値を調整する

In [9]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):        
        
        # 推論と損失の計算
        
        ## モデルからロジットを求める
        pred = model(X)
        
        ## ロジットと正解ラベルを損失関数に与えて, 損失値(スカラ―)を得る
        loss = loss_fn(pred, y)
        
        ## 勾配をリセット
        optimizer.zero_grad()
        
        ## backpropで損失関数に対する勾配を求める
        loss.backward()
        
        ## 勾配を使用して, パラメータの値を調整
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
            
    test_loss /= size
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [10]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.306533  [    0/60000]
loss: 2.300227  [ 6400/60000]
loss: 2.281330  [12800/60000]
loss: 2.275303  [19200/60000]
loss: 2.260060  [25600/60000]
loss: 2.252860  [32000/60000]
loss: 2.260581  [38400/60000]
loss: 2.235570  [44800/60000]
loss: 2.233417  [51200/60000]
loss: 2.219148  [57600/60000]
Test Error: 
 Accuracy: 47.9%, Avg loss: 0.034661 

Epoch 2
-------------------------------
loss: 2.222991  [    0/60000]
loss: 2.204427  [ 6400/60000]
loss: 2.167656  [12800/60000]
loss: 2.177789  [19200/60000]
loss: 2.119698  [25600/60000]
loss: 2.126154  [32000/60000]
loss: 2.142106  [38400/60000]
loss: 2.097288  [44800/60000]
loss: 2.094998  [51200/60000]
loss: 2.068624  [57600/60000]
Test Error: 
 Accuracy: 53.2%, Avg loss: 0.032227 

Epoch 3
-------------------------------
loss: 2.091789  [    0/60000]
loss: 2.043935  [ 6400/60000]
loss: 1.977867  [12800/60000]
loss: 2.002035  [19200/60000]
loss: 1.888289  [25600/60000]
loss: 1.923173  [32000/600

# モデルの重みの保存と読み込み

- モデルは学習したパラメータを内部に状態辞書(state_dict)として保持している
- モデルパラメータの値は`torch.save`を使用することで保存することができる
- **推論前にはDropoutやBatchNormをevaluationモードに切り替えるため, 推論前にはmodel.eval()を実行する必要がある**

In [5]:
import torch
import torch.onnx as onnx
import torchvision.models as models

model = models.vgg16(pretrained=True)
torch.save(model.state_dict(), 'model_weights.pth')

In [7]:
del model
model = models.vgg16()
model.load_state_dict(torch.load('model_weights.pth'))

# 推論モードへの切り替え
model.eval()

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

## モデルの形ごと保存・読み込む方法

- モデルの重みをロードする場合, 先にモデルのインスタンスを用意しておかなければならない
- モデルクラスの構造も一緒に保持したい場合、`torch.load()`を用いることでモデルインスタンスを得ることができる
  - ただしその場合は, `model.state_dict()`ではなく, モデルインスタンス自体を`torch.save()`に指定しなければならない

In [11]:
torch.save(model, 'model.pth')
model = torch.load('model.pth')
model

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

## ONNX形式でのモデル出力

- ONNX形式でモデルを保存するには`torch.onnx.export()`を使用する
- PyTorchの計算グラフは動的に生成されるため, 出力を行うには, 計算グラフを実行しておく必要がある
  - すなわち, テスト用の適切なテンソルサイズの入力変数を用意して, それをモデルで推論しておく必要がある

In [12]:
# 入力画像のダミーとして, 同じ形状のゼロ行列を作成
input_image = torch.zeros((1, 3, 244, 244))
onnx.export(model, input_image, 'model.onnx')